## Exploratory analysis 



### Load the dataset

In [6]:
import json
with open('l1.json', 'r') as f:
    l1_dict=json.load(f)
l1=list(l1_dict.keys())
neg_adj=list(l1_dict.values())
with open('l2.json', 'r') as f:
    l2_dict=json.load(f)
l2=list(l2_dict.keys())
with open('l3.json', 'r') as f:
    l3_dict=json.load(f)
l3=list(l3_dict.keys())
pos_adj=list(l3_dict.values())
with open('l4.json', 'r') as f:
    l4_dict=json.load(f)
l4=list(l4_dict.keys())
with open('l5.json', 'r') as f:
    l5_dict=json.load(f)
l5=list(l5_dict.keys())

### Setting everything up for Trasnformer lens library

In [7]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
DEBUG_MODE = False
IN_COLAB = False
print("Running as a Jupyter notebook - intended for development only!")
from IPython import get_ipython
ipython = get_ipython()
# Code to automatically update the HookedTransformer code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

pio.renderers.default = "vscode"
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader
from jaxtyping import Float, Int

from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
torch.set_grad_enabled(False)
from neel_plotly import line, imshow, scatter
import transformer_lens.patching as patching
model = HookedTransformer.from_pretrained("gpt2-small")
prompts = ['The day is bright but the night is']
not_id=model.to_single_token(" not")
bright_id=model.to_single_token(" bright")
clean_tokens = model.to_tokens(prompts)
print("Clean string 0", model.to_string(clean_tokens[0]))
logits,cache=model.run_with_cache(clean_tokens)
id=logits[...,-1,:].argmax()
model.to_string(id)

import plotly.io as pio
pio.renderers.default = "vscode"


def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)


def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

Running as a Jupyter notebook - intended for development only!
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_55566/1326878066.py:9: DeprecationWarning:

`magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).

/tmp/ipykernel_55566/1326878066.py:10: DeprecationWarning:

`magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: d5557f85-ee9d-4292-ba4f-414d0533f9b3)')' thrown while requesting HEAD https://huggingface.co/gpt2/resolve/main/config.json
Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer
Clean string 0 <|endoftext|>The day is bright but the night is


## Which heads directly write to the output?

Which attention heads directly affect the model's output:

(The heads writting in the resiudual stream at the END position, in a direction that has high dot product with the logit difference)

$W_U$ The unembedding matrix

$\overline{LN}$ the layer norm

We search for heads (i,j) such that
$$
\lambda_{i,j}:=\mathbb{E}_{x\sim \mathbb{P}_{BS} }[\langle \overline{LN} \circ h_{i,j}(X),W_U[a]-W_U[b]\rangle]
$$